In [1]:
import os, typing, json

from conflux_web3 import Web3
from conflux_web3.dev import get_mainnet_web3
from dotenv import load_dotenv

load_dotenv()

class Metadata(typing.TypedDict):
    abi: typing.Any
    bytecode: typing.Any

def get_contract_metadata(path_: str) -> Metadata:
    with open(path_) as f:
        metadata = json.load(f)
        return {
            "abi": metadata["abi"],
            "bytecode": metadata["bytecode"]["object"]
        }

In [9]:
w3 = get_mainnet_web3()
acct = w3.account.from_key(os.environ["SK"])
w3.cfx.default_account = acct

Registry = w3.cfx.contract(**get_contract_metadata("./out/ERC6551Registry.sol/ERC6551Registry.json"))
registry_address = Registry.constructor().transact().executed()["contractCreated"]
if registry_address is None:
    raise Exception("unexpected result")
print(f"registry address: {registry_address}")
registry = Registry(
    registry_address
)

registry address: cfx:ach1zpmjyv17arzdjx9fgbt4cw6uns0ntuwre8sdsy


In [10]:
Implementation = w3.cfx.contract(**get_contract_metadata("./out/ERC6551AccountUpgradeable.sol/ERC6551AccountUpgradeable.json"))
simple_impl_addr = Implementation.constructor().transact().executed()["contractCreated"]
if simple_impl_addr is None:
    raise Exception("unexpected result")
# print(f"implementation result: {simple_impl_addr}")
# acct_impl = Implementation(simple_impl_addr)
simple_impl_addr

'cfx:acd9jgz4d5as156ef08r5mdgux28y51yz639nr5s78'

In [11]:
admin_control = w3.cfx.contract(name="AdminControl", with_deployment_info=True)
admin_control.functions.setAdmin(registry.address, w3.address.zero_address()).transact().executed()
admin_control.functions.setAdmin(simple_impl_addr, w3.address.zero_address()).transact().executed()


AttributeDict({'transactionHash': HexBytes('0x104fbd92baf35dc1825413a470f80429ad58dd09a0a0718c9ac07565438abb7c'),
 'index': 0,
 'blockHash': HexBytes('0xae0900e5691ed42443bce55b5c3bcad2fe522d7b78d0a8faa5876f6c6deded18'),
 'epochNumber': 78314716,
 'from': 'cfx:aanthezva9vuyrpsmb15sf9xt97f8vduky1k1vwka5',
 'to': 'cfx:aaejuaaaaaaaaaaaaaaaaaaaaaaaaaaaaa2mhjju8k',
 'gasUsed': 27808,
 'gasFee': 27808000000000 Drip,
 'contractCreated': None,
 'logs': [],
 'logsBloom': HexBytes('0x00000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000'),
 'sta